In [1]:
import pandas as pd
import numpy as np 
import scipy as sp
import statsmodels.formula.api as sm 
from statsmodels.iolib.summary2 import summary_col
import datetime as dt
from datetime import datetime
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (16.0, 8.0)

# 1. Asset Pricing Models

## 1.1. CAPM Regression Framework

$$ r - r_f = \alpha + \beta_m (r_m - r_f) + \epsilon $$

## 1.2. FF3 Regression Framework

$$ r - rf = \alpha + \beta_m(r_m - r_f) + \beta_{s}SMB + \beta_{v} HML + \epsilon $$

$SMB$ = return spread of small minus large stocks (size).

$HML$ = The return of cheap minus expensive stocks (value).

## 1.3. FF5 Regression Framework

$$r - rf = \alpha + \beta_m(r_m - r_f) + \beta_{s}SMB + \beta_{v}HML + \beta_{p}RMW + \beta_{i}CMA +  \epsilon $$



$RMW$ = The return spread of the most profitable firms minus the least profitable (profit).

$CMA$ = The return spread of firms that invest conservatively minus aggressively (investment).

In [5]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

class Stock:
    def __init__(self, ticker, start=None, end=None):

        self.ticker = ticker

        try:
            self._ticker = yf.Ticker(self.ticker)

            if not (start or end):
                self.df = self.df_ = self._ticker.history(period='max', auto_adjust=True)
                
            else:
                self.df = self.df_ = self._ticker.history(start=start, end=end, auto_adjust=True)

        except Exception as err:
            print(err)
            
            
def adj_data(sym):
    df = Stock(sym).df
    ohlc = df[['Close']]
    ohlc =ohlc.rename(columns={'Close': sym})
    ohlc.name = sym 
    return ohlc       

In [11]:
def to_return(prices,retType='simple'):
    if retType == 'simple':
        ret = (prices/prices.shift(1))-1
    else:
        ret = np.log(prices/prices.shift(1))
    return ret

In [21]:
def factor_regression(df_stk):
    stkName = df_stk.name
    import pandas_datareader.data as web 
    df_factors = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench')[0]
    df_factors.rename(columns={'Mkt-RF': 'MKT'}, inplace=True)
    df_factors['MKT'] = df_factors['MKT']/100
    df_factors['SMB'] = df_factors['SMB']/100
    df_factors['HML'] = df_factors['HML']/100
    df_factors['RMW'] = df_factors['RMW']/100
    df_factors['CMA'] = df_factors['CMA']/100
    df_stock_factor = pd.merge(df_stk,df_factors,left_index=True,right_index=True)
    df_stock_factor['XsRet'] = df_stock_factor[stkName] - df_stock_factor['RF'] 
    
    CAPM = sm.ols(formula = 'XsRet ~ MKT', data=df_stock_factor).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    FF3 = sm.ols( formula = 'XsRet ~ MKT + SMB + HML', data=df_stock_factor).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    FF5 = sm.ols( formula = 'XsRet ~ MKT + SMB + HML + RMW + CMA', data=df_stock_factor).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    
    CAPMtstat = CAPM.tvalues
    FF3tstat = FF3.tvalues
    FF5tstat = FF5.tvalues
    
    CAPMcoeff = CAPM.params
    FF3coeff = FF3.params
    FF5coeff = FF5.params
    
    CAPMpredi = CAPM.predict()
    FF3predi = FF3.predict()
    FF5predi = FF5.predict()
    
    CAPMresid = CAPM.resid
    FF3resid = FF3.resid
    FF5resid = FF5.resid

    results_df = pd.DataFrame({'CAPMcoeff':CAPMcoeff,'CAPMtstat':CAPMtstat,
                               'FF3coeff':FF3coeff, 'FF3tstat':FF3tstat,
                               'FF5coeff':FF5coeff, 'FF5tstat':FF5tstat},
    index = ['Intercept', 'MKT', 'SMB', 'HML', 'RMW', 'CMA'])
    
    df_stock_factor['CAPM'] = CAPMpredi
    df_stock_factor['CAPMResiduals'] = CAPMresid
    # Fama-French 3 Factor Model (FF3)
    df_stock_factor['FF3'] = FF3predi
    df_stock_factor['FF3Residuals'] = FF3resid 
    # Fama-French 5 Factor Model (FF5)
    df_stock_factor['FF5'] = FF5predi
    df_stock_factor['FF5Residuals'] = FF5resid
    
    dfoutput = summary_col([CAPM, FF3, FF5],stars=True,float_format='%0.4f',
                  model_names=['CAPM','FF3','FF5'],
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'Adjusted R2':lambda x: "{:.4f}".format(x.rsquared_adj)}, 
                             regressor_order = ['Intercept', 'MKT', 'SMB', 'HML', 'RMW', 'CMA'])
    
    results_df.to_csv('stats/stats_'+stkName+'.csv')
    df_stock_factor.to_csv('apm/'+stkName+'.csv')

    return print(dfoutput)

In [30]:
df = adj_data("AMD")
sym = df.name
data = to_return(df,retType='simple')
data.name = sym
data.dropna(inplace=True)
print('==============================================')
print(sym+' - Risk Factor Regression')
print('==============================================')
factor_regression(data)
print('                                              ')

AMD - Risk Factor Regression

                  CAPM      FF3        FF5    
----------------------------------------------
Intercept      -0.0021** -0.0025*** -0.0025***
               (0.0009)  (0.0009)   (0.0009)  
MKT            1.4976*** 1.5454***  1.5140*** 
               (0.0837)  (0.0749)   (0.0722)  
SMB                      0.3801***  0.3297**  
                         (0.1291)   (0.1378)  
HML                      -0.9941*** -0.8556***
                         (0.0964)   (0.1123)  
RMW                                 -0.1999   
                                    (0.2008)  
CMA                                 -0.4181   
                                    (0.2956)  
R-squared      0.2450    0.2971     0.2990    
R-squared Adj. 0.2443    0.2954     0.2961    
N              1228      1228       1228      
Adjusted R2    0.2443    0.2954     0.2961    
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01
                                              


In [6]:
amd = pd.read_csv("apm/amd.csv").set_index("Date")

In [7]:
amd.describe()

,AMD,MKT,SMB,HML,RMW,CMA,RF,XsRet,CAPM,CAPMResiduals,FF3,FF3Residuals,FF5,FF5Residuals
count,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1.228000e+03,1228.000000,1.228000e+03,1228.000000,1.228000e+03
mean,0.003183,0.000695,0.000099,-0.000266,0.000028,-0.000116,0.004280,-0.001097,-0.001097,-9.718972e-19,-0.001097,-1.606409e-15,-0.001097,-3.006749e-15
std,0.037117,0.012333,0.007030,0.009343,0.004343,0.003960,0.003419,0.037319,0.018470,3.242763e-02,0.020341,3.128771e-02,0.020406,3.124582e-02
min,-0.242291,-0.120000,-0.045800,-0.048900,-0.018800,-0.022600,0.000000,-0.245291,-0.181853,-2.436030e-01,-0.183609,-2.434228e-01,-0.186594,-2.440446e-01
25%,-0.015970,-0.003000,-0.003625,-0.004400,-0.002500,-0.002400,0.001000,-0.021188,-0.006630,-1.716683e-02,-0.008513,-1.637696e-02,-0.008491,-1.651257e-02
50%,0.001917,0.000900,-0.000200,-0.000600,-0.000100,-0.000200,0.004000,-0.002021,-0.000789,-2.566285e-03,-0.000100,-2.645821e-03,-0.000128,-2.338174e-03
75%,0.020731,0.005900,0.003800,0.003600,0.002500,0.001900,0.007000,0.016723,0.006699,1.344973e-02,0.008471,1.321352e-02,0.008442,1.313741e-02
max,0.199481,0.093400,0.057300,0.067000,0.017000,0.024300,0.010000,0.189481,0.137742,1.783853e-01,0.117860,1.866163e-01,0.116606,1.861598e-01
